In [1]:
"""
Notebook for training the embedding model for the Lorenz system.
=====
Distributed by: Notre Dame SCAI Lab (MIT Liscense)
- Associated publication:
url: https://arxiv.org/abs/2010.03957
doi: 
github: https://github.com/zabaras/transformer-physx
=====
"""
!nvidia-smi

zsh:1: command not found: nvidia-smi


# Environment Setup

Use pip to install from [PyPI](https://pypi.org/project/trphysx/).

In [2]:
%ls

data/                           train_lorenz_enn.py
outputs/                        train_lorenz_transformer.ipynb
predict_lorenz.py               train_lorenz_transformer.py
train_lorenz_enn.ipynb


Now lets download the training and validation data for the lorenz system. Info on wget from [Google drive](https://stackoverflow.com/questions/37453841/download-a-file-from-google-drive-using-wget). This will eventually be update to zenodo repo.

In [3]:
!mkdir data

mkdir: data: File exists


# Transformer-PhysX Lorenz System

Train the embedding model.
First import necessary modules from trphysx. 

In [4]:
import sys, os
import logging

import paddle
from paddle.optimizer.lr import ExponentialDecay

from trphysx.config.configuration_auto import AutoPhysConfig
from trphysx.embedding.embedding_auto import AutoEmbeddingModel
from trphysx.embedding.training import *

Training arguments.

In [5]:
argv = []
argv = argv + ["--exp_name", "lorenz"]
argv = argv + ["--training_h5_file", "./data/lorenz_training_rk.hdf5"]
argv = argv + ["--eval_h5_file", "./data/lorenz_valid_rk.hdf5"]
argv = argv + ["--batch_size", '512']
argv = argv + ["--block_size", "16"]
argv = argv + ["--n_train", "2048"]
argv = argv + ["--n_eval", "64"]
argv = argv + ["--epochs", "100"]

In [6]:
args = EmbeddingParser().parse(args=argv)  
# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO)

use_cuda = False
if paddle.is_compiled_with_cuda():
    use_cuda = True
args.device = paddle.set_device("gpu:0" if use_cuda else "cpu")

## Initializing Datasets and Models

Now we can use the auto classes to initialized the predefined configs, dataloaders and models. This may take a bit!

In [8]:
 # Load transformer config file
config = AutoPhysConfig.load_config(args.exp_name)
dataloader = AutoDataHandler.load_data_handler(args.exp_name)

# Set up data-loaders
training_loader = dataloader.createTrainingLoader(
    args.training_h5_file, 
    block_size=args.block_size, 
    stride=args.stride, 
    ndata=args.n_train, 
    batch_size=args.batch_size)
testing_loader = dataloader.createTestingLoader(
    args.eval_h5_file, 
    block_size=32, 
    ndata=args.n_eval, 
    batch_size=8)

# Set up model
model = AutoEmbeddingModel.init_trainer(args.exp_name, config).to(args.device)
mu, std = dataloader.norm_params
if args.epoch_start > 1:
  model.load_model(args.ckpt_dir, args.epoch_start)

count = 0
for name, param in model.named_parameters():
    print(name, param.dtype,param.stop_gradient)
    count += param.numel()

embedding_model.kMatrixDiag paddle.float32 False
embedding_model.kMatrixUT paddle.float32 False
embedding_model.observableNet.0.weight paddle.float32 False
embedding_model.observableNet.0.bias paddle.float32 False
embedding_model.observableNet.2.weight paddle.float32 False
embedding_model.observableNet.2.bias paddle.float32 False
embedding_model.observableNet.3.weight paddle.float32 False
embedding_model.observableNet.3.bias paddle.float32 False
embedding_model.recoveryNet.0.weight paddle.float32 False
embedding_model.recoveryNet.0.bias paddle.float32 False
embedding_model.recoveryNet.2.weight paddle.float32 False
embedding_model.recoveryNet.2.bias paddle.float32 False


Initialize optimizer and scheduler. Feel free to change if you want to experiment.

In [ ]:
scheduler = ExponentialDecay(
    learning_rate=args.lr * 0.995 ** (args.epoch_start - 1), gamma=0.995
)

optimizer = paddle.optimizer.Adam(
    parameters=model.parameters(),
    learning_rate=scheduler,
    weight_decay=1e-8,
)

## Training the Embedding Model

Train the model. No visualization here, just boring numbers. This notebook only trains for 100 epochs for brevity, feel free to train longer. The test loss here is only the recovery loss MSE(x - decode(encode(x))) and does not reflect the quality of the Koopman dynamics.

In [ ]:
trainer = EmbeddingTrainer(model, args, (optimizer, scheduler))
trainer.train(training_loader, testing_loader)

/Users/puqing/opt/miniconda3/lib/python3.9/site-packages/paddle/fluid/dygraph/math_op_patch.py:275: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float64, but right dtype is paddle.float32, the right dtype will convert to paddle.float64
  warnings.warn(
/Users/puqing/opt/miniconda3/lib/python3.9/site-packages/astroid/node_classes.py:90: DeprecationWarning: The 'astroid.node_classes' module is deprecated and will be replaced by 'astroid.nodes' in astroid 3.0.0
  warnings.warn(


ValueError: (InvalidArgument) Leaf Tensor () that doesn't stop gradient can't use inplace strategy.
  [Hint: Expected egr::egr_utils_api::IsLeafTensor(self->tensor) && !egr::EagerUtils::autograd_meta(&self->tensor)->StopGradient() == false, but received egr::egr_utils_api::IsLeafTensor(self->tensor) && !egr::EagerUtils::autograd_meta(&self->tensor)->StopGradient():1 != false:0.] (at /home/Paddle/paddle/fluid/pybind/eager_method.cc:1063)


Check your Google drive for checkpoints.